In [3]:
import pandas as pd
import os
import glob
import numpy as np
import ipyvuetify as v
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox, HBox, HTML
from ipyleaflet import AwesomeIcon, Marker, Map, Icon, Popup, basemaps, Polyline
from bqplot import Axis, Figure, Lines, LinearScale
import bqplot as bq
import matplotlib.pyplot as plt
import matplotlib as mpl
import io
from scipy.optimize import curve_fit
from stl import mesh
import pythreejs as p3

In [4]:
# TO DO : add title, image (inav, x-lite, moi qui vole sur l'aile)
# sort value before fitting ? it will destroy time slider interaction ?
# add HD video visualisation

In [5]:
widgets.HTML('<center> <img src="images/gaju_fpv.gif" align="middle" class="center"> </center>')

HTML(value='<center> <img src="images/gaju_fpv.gif" align="middle" class="center"> </center>')

In [6]:
def get_rssi(RSSI, dist):
    [a,b] = np.polyfit(np.log(dist), RSSI, 1)
    rssi_time = a * np.log(dist) + b
    rssi_dist = a * np.log(np.sort(dist)) + b
    return rssi_time, rssi_dist

def get_max(data, data_type='str'):
    max_dist  = np.max(data['Dist(m)'])
    max_speed = np.max(data['GSpd(kmh)'])
    max_alt   = np.max(data['Alt(m)'])
    if data_type=="str":
        max_dist  = str(max_dist)
        max_speed = str(max_speed)
        max_alt   = str(max_alt)
    return max_dist, max_speed, max_alt

In [7]:
files = glob.glob('logs/*.csv')

In [8]:
# Get max stats
max_dist_tot, max_speed_tot, max_alt_tot = 0, 0, 0
for file in files:
    data = pd.read_csv(file, sep=';', parse_dates=[0,1] , infer_datetime_format=True)
    new_dist, new_speed, new_alt = get_max(data, data_type='int')
    max_dist_tot  = np.max([max_dist_tot,new_dist])
    max_speed_tot = np.max([max_speed_tot,new_speed])
    max_alt_tot   = np.max([max_alt_tot,new_alt])
max_dist_tot  = str(max_dist_tot)
max_speed_tot = str(max_speed_tot)
max_alt_tot   = str(max_alt_tot)

/tmp/ipykernel_105793/3922535483.py:4: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv(file, sep=';', parse_dates=[0,1] , infer_datetime_format=True)
/tmp/ipykernel_105793/3922535483.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(file, sep=';', parse_dates=[0,1] , infer_datetime_format=True)


In [9]:
class TelemetryData:
    def __init__(self):
        self.dist, self.rssi = [], []
        self.lon, self.lat   = [], []
        self.time = []
        self.rssi_time, self.rssi_dist = [], []
        self.dist, self.speed, self.alt = 0, 0, 0
        self.max_dist, self.max_speed, self.max_alt = 0, 0, 0
        self.data = []
        self.hdg = []
    def get_data(self, file):
        self.data = pd.read_csv(file, sep=';', parse_dates=[0,1] , infer_datetime_format=True)
        #self.data = data.interpolate(method='nearest', column=['GPS(lat)','GPS(lon)']).ffill().bfill()
        self.data  = self.data[~(self.data['GPS(lat)'].isnull())]
        dist, rssi = self.data['Dist(m)'], self.data['RSSI(dB)']
        filt = (rssi != 0) & (dist != 0)
        self.lon  = self.data['GPS(lon)']
        self.lat  = self.data['GPS(lat)']
        self.time = np.arange(len(self.data['Time']))
        self.dist_filt = dist[filt]
        if filt.sum() > 0:
            self.rssi_time, self.rssi_dist = get_rssi(rssi[filt], dist[filt])
        self.speed, self.alt, self.dist = self.data['GSpd(kmh)'], self.data['Alt(m)'], dist
        self.max_dist, self.max_speed, self.max_alt = get_max(self.data)
        self.hdg = self.data['Hdg(@)']

In [10]:
# Initialize Data
tlm = TelemetryData()
tlm.get_data(files[0])

/tmp/ipykernel_105793/4272251496.py:12: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  self.data = pd.read_csv(file, sep=';', parse_dates=[0,1] , infer_datetime_format=True)
/tmp/ipykernel_105793/4272251496.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.data = pd.read_csv(file, sep=';', parse_dates=[0,1] , infer_datetime_format=True)


In [11]:
# 3D MODEL
stl_mesh = mesh.Mesh.from_file('3d/fixed_wing.stl')
# Create a pythreejs geometry directly from the STL data
faces, face_normals = stl_mesh.vectors, stl_mesh.normals
vertices, normals, indices = [], [], []
for i, face in enumerate(faces):
    base_index = len(vertices)
    for vertex in face:
        vertices.append(vertex)
        normals.append(face_normals[i])  # Use the face normal for all vertices
    # Add indices for this face
    indices.extend([base_index, base_index + 1, base_index + 2])

# Convert to numpy arrays
vertices = np.array(vertices, dtype=np.float32)
normals = np.array(normals, dtype=np.float32)
indices = np.array(indices, dtype=np.uint32)

# Create a buffer geometry
geometry = p3.BufferGeometry(
    attributes={
        'position': p3.BufferAttribute(vertices, normalized=False),
        'normal': p3.BufferAttribute(normals, normalized=False),
        'index':  p3.BufferAttribute(indices, normalized=False)
    }
)

# Create a material with good lighting properties
material = p3.MeshPhongMaterial(
    color='#404040',  # Blue color
    emissive='#000000',
    specular='#000000',
    shininess=0
)

# Create the mesh
mesh_3d = p3.Mesh(geometry=geometry, material=material)
center = np.mean(vertices, axis=0)
size = np.max(np.abs(vertices - center))

# Create a scene
scene = p3.Scene()
scene.add(mesh_3d)
main_light = p3.DirectionalLight(color='white', position=[size, size, size], intensity=0.3)
fill_light = p3.DirectionalLight(color='white', position=[-size, size, -size], intensity=0.3)
back_light = p3.DirectionalLight(color='white', position=[0, -size, 0], intensity=0.3)
ambient_light = p3.AmbientLight(intensity=0.4)
scene.add(main_light)
scene.add(fill_light)
scene.add(back_light)
scene.add(ambient_light)

# Create a camera
camera = p3.PerspectiveCamera(
    position=[center[0], center[1], center[2]],
    up=[0, 1, 0],
    fov=40
)
orbit_controls = p3.OrbitControls(controlling=camera)

# Create a renderer
renderer = p3.Renderer(
    camera=camera,
    scene=scene,
    controls=[orbit_controls],
    width=600,
    height=400,
    antialias=True
)

In [12]:
# MAIN COMPONENTS

In [13]:
out = widgets.Output()

In [14]:
# left - files
uploader    = widgets.FileUpload(accept='.csv', multiple=True, layout=Layout(width='14vw'))
file_select = widgets.Dropdown(options=[os.path.basename(f) for f in files], layout=Layout(width='14vw'))
#if file_exist:
#video       = widgets.Video.from_file("videos/Big.Buck.Bunny.mp4", autoplay=False,layout=Layout(width='14vw'))
ar_wing    = widgets.HTML('<center> <img src="images/ar_wing.png" align="middle" class="center"> </center>')
dist_curr  = widgets.HTML(value='<b>Distance (m)</b> : '+str(tlm.dist.iloc[5]))
speed_curr = widgets.HTML(value='<b>Speed (km/h)</b> : '+str(tlm.speed.iloc[5]))
alt_curr   = widgets.HTML(value='<b>Altitude (m)</b> : '+str(tlm.alt.iloc[5]))
infos      = VBox(children=[dist_curr, speed_curr, alt_curr], layout=Layout(align_items='center', border='solid 1px black', flex_direction='column', align_content='flex-end'))
pitot      = widgets.ToggleButton(value=False, description='Check Pitot', button_style='', icon='magic', layout=Layout(width='auto', align_items='center', flex_direction='column', align_content='flex-end'))
dist_max  = widgets.HTML(value="<b>Distance (m)</b> : "+tlm.max_dist+" / "+max_dist_tot)
speed_max = widgets.HTML(value="<b>Speed (km/h)</b> : "+tlm.max_speed+" / "+max_speed_tot)
alt_max   = widgets.HTML(value="<b>Altitude (m)</b> : "+tlm.max_alt+" / "+max_alt_tot)
trophee   = widgets.HTML('<center> <img src="images/trophy.jpg" align="middle" class="center"> </center>')
records   = VBox(children=[dist_max, speed_max, alt_max], layout=Layout(width='auto', height='10vh', align_items='center', border='solid 1px black'))
left      = VBox(children=[uploader, file_select, ar_wing, pitot, trophee, records])#,video])

# center - map and time slider
my_map      = Map(basemap=basemaps.Esri.WorldImagery, center=(np.mean(tlm.lat), np.mean(tlm.lon)), zoom=16)
icon_home   = AwesomeIcon(name='home', icon_color="white", marker_color='black')
marker_home = Marker(icon=icon_home, location=(tlm.lat.iloc[0], tlm.lon.iloc[0]), draggable=False)
icon_plane  = Icon(icon_url='images/icon_airplane.png')
marker_plane = Marker(icon=icon_plane, location=(tlm.lat.iloc[-1], tlm.lon.iloc[-1]), rotation_angle=tlm.hdg.iloc[-1], rotation_origin="bottom center", draggable=False)
marker_plane.popup = infos
line_map    = Polyline(locations=[(x, y) for x, y in zip(tlm.lat, tlm.lon)], color = "red", fill=False, weight=2)
time_slider = widgets.IntSlider(value=len(tlm.time), max=len(tlm.time), layout=Layout(flex='1.5 1 0%', width='auto'))
play_button = widgets.Play(max=len(tlm.time))
animation   = widgets.HBox([play_button, time_slider]) 
my_map.add_layer(marker_home)
my_map.add_layer(line_map)
my_map.add_layer(marker_plane)
widgets.jslink((play_button, 'value'), (time_slider, 'value'))
center = VBox(children=[my_map, animation], layout=Layout(flex='1.5 1 0%', width='auto', height='80vh', align_content='center', justify_content="center"))

# right - rssi graphs and records
x_var = widgets.Dropdown(options=tlm.data.columns, description='X axis :', value = 'Dist(m)', layout=Layout(width='12vw'))
y_var = widgets.Dropdown(options=tlm.data.columns, description='Y axis :', value = 'RSSI(dB)', layout=Layout(width='12vw'))
fit_button = widgets.ToggleButton(value=False, description='Fit', button_style='', icon='bolt', layout=Layout(width='5vw'))
xy_var  = widgets.HBox([x_var, y_var, fit_button], layout=Layout(width='auto', height='auto', align_content='center', justify_content="center")) 
xs, ys  = bq.LinearScale(), bq.LinearScale()
y_axis  = bq.Axis(scale=ys, label='RSSI(dB)', grid_lines='solid', orientation='vertical' )
x_axis  = bq.Axis(scale=xs, label='Dist(m)', grid_lines='solid')
tooltip = bq.Tooltip(fields=['x', 'y'], labels=['dist','rssi'])
points  = bq.Scatter(x=tlm.data['Dist(m)'], y=tlm.data['RSSI(dB)'], scales={'x':xs,'y':ys}, default_size=20, labels=['data'], display_legend=True, tooltip=tooltip, unhovered_style={'opacity': 0.5})
line    = bq.Lines(x=np.sort(tlm.dist_filt), y=tlm.rssi_dist, scales={'x': xs, 'y': ys}, colors=['red'], labels=['fit'], display_legend=True) 
fig     = bq.Figure(marks=[points, line], axes=[x_axis, y_axis], layout=Layout(width='30vw', height = '45vh'))
right     = VBox(children=[xy_var, fig, renderer], layout=Layout(flex='1 1 0%', width='auto', height='80vh', align_items='center', flex_direction="column"))

In [15]:
# INTERACTION

In [16]:
# function for updating all data : used withe selection and upload widgets
@out.capture()
def update_all(file):
    with out:
        tlm.get_data(file)
        x_var.options = tlm.data.columns
        y_var.options = tlm.data.columns
        x_var.value = 'Dist(m)'
        y_var.value = 'RSSI(dB)'
        #tlm.data = tlm.data.sort_values(x_var.value)
        time_slider.max = len(tlm.time)
        # update map
        my_map.center   = np.mean(tlm.lat), np.mean(tlm.lon)
        my_map.layers[1].location = (tlm.lat.iloc[0], tlm.lon.iloc[0])
        my_map.layers[2].locations = [(x, y) for x, y in zip(tlm.lat, tlm.lon)]
        # update graph
        points.x = tlm.data['Dist(m)']
        line.x   = np.sort(tlm.dist_filt)
        points.y = tlm.data['RSSI(dB)']
        line.y   = tlm.rssi_dist
        # update stats
        dist_max.value  = "<b>Distance (m)</b> : "+tlm.max_dist+" / "+max_dist_tot
        speed_max.value = "<b>Speed (km/h)</b> : "+tlm.max_speed+" / "+max_speed_tot
        alt_max.value   = "<b>Altitude (m)</b> : "+tlm.max_alt+" / "+max_alt_tot

In [17]:
@out.capture()
def file_change(change):
    with out:
        update_all('logs/'+change['new'])
file_select.observe(file_change, names='value')

@out.capture()
def upload_change(change):
    with out:
        uploaded_file = change['new']
        file = io.BytesIO(next(iter(uploaded_file.values()))['content'])
        update_all(file)
uploader.observe(upload_change, names='value')

@out.capture()
def xaxis_change(change):
    with out:
        if (change['new'] != 'Dist(m)') or (change['new'] != 'Time'):
            line.y   = []
        #tlm.data = tlm.data.sort_values(change['new'])
        points.x = tlm.data[change['new']]
        points.y = tlm.data[y_var.value]
        x_axis.label = change['new']
        tooltip.labels[0] = change['new']
        #line.marker = 'circle'
        line.line_style = 'solid'
x_var.observe(xaxis_change, names='value')

@out.capture()
def yaxis_change(change):
    with out:
        if (change['new'] != 'Rssi(dB)'):
            line.y   = []        
        points.y = tlm.data[change['new']]
        y_axis.label = change['new']
        tooltip.labels[1] = change['new']
y_var.observe(yaxis_change, names='value')

@out.capture()
def time_change(change):
    with out:
        last_time = change['new']
        my_map.layers[2].locations = [(x, y) for x, y in zip(tlm.lat[:last_time], tlm.lon[:last_time])]
        my_map.layers[3].location = (tlm.lat.iloc[last_time-1], tlm.lon.iloc[last_time-1])
        my_map.layers[3].rotation_angle = tlm.hdg.iloc[last_time-1]
        points.x = tlm.data[x_var.value].iloc[:last_time-1]
        points.y = tlm.data[y_var.value].iloc[:last_time-1]
        dist_curr.value  = '<b>Distance (m)</b> : '+str(tlm.dist.iloc[last_time-1])
        speed_curr.value = '<b>Speed (km/h)</b> : '+str(tlm.speed.iloc[last_time-1])
        alt_curr.value   = '<b>Altitude (m)</b> : '+str(tlm.alt.iloc[last_time-1])
time_slider.observe(time_change, names='value')

In [18]:
# DISPLAY

In [19]:
display(HBox(children=[left, center, right]), out)

Output()

In [20]:
@out.capture()
def pitot_change(change):
    with out:
        if 'ASpd(kmh)' in tlm.data.keys():
            # original
            plt.figure(figsize=(20,5))
            plt.subplot(131)
            plt.plot(tlm.data['ASpd(kmh)'], label='ASpd(kmh)')
            plt.plot(tlm.data['GSpd(kmh)'], label='GSpd(kmh)')
            plt.xlabel('time',fontsize=15)
            plt.legend()
            # fit
            d = tlm.data['ASpd(kmh)']
            x = np.arange(d.max())
            seuil = 15
            def func(x, a):
                return a * x
            a, pcov = curve_fit(func, tlm.data['ASpd(kmh)'][d>seuil], tlm.data['GSpd(kmh)'][d>seuil])
            plt.subplot(132)
            plt.plot(tlm.data['ASpd(kmh)'], tlm.data['GSpd(kmh)'],'.')
            plt.plot(tlm.data['ASpd(kmh)'], func(tlm.data['ASpd(kmh)'], *a))
            plt.title('scale={:.2f}'.format(a[0]))
            plt.subplot(133)
            plt.plot(a[0]*tlm.data['ASpd(kmh)'], label='ASpd(kmh)')
            plt.plot(tlm.data['GSpd(kmh)'], label='GSpd(kmh)')
            plt.xlabel('time',fontsize=15)
            plt.legend()
            plt.show()
pitot.observe(pitot_change, names='value')

In [21]:
# video

In [22]:
#video = widgets.Video.from_file("../../videos/RC_VID_0015.MOV", autoplay=False, layout=Layout(width='14vw'))